In [1]:
import re
import requests
import os
from datetime import datetime

from bs4 import BeautifulSoup, Comment
from bs4.element import Tag

import pandas as pd
from tqdm import tqdm

In [2]:
def get_description(breed_soup):
#     print('breed_soup', breed_soup);
    try:
        description = breed_soup.find(
                'p', class_='breed-page__about__read-more__text'
        ).get_text().strip()
    except:
        description = ''
        
#     print('description', description)
    
    # Removing weird characters
    # Probably not exhaustive
    description = description.replace(
        '\n', '').replace('\u200b', '').replace('\xa0', ' ')
    
#     print('description', description)
    return description

In [3]:
def get_temperament(breed_soup):
    temperment_class = 'breed-page__intro__temperment'
    try:
        return breed_soup.find(
            'p', class_=temperment_class
        ).get_text().replace(' / ', ',')
    except:
        return ''

In [4]:
def general_regex(text, var, mul=1):
    reg = re.compile('(\d+\.?\d*)')
    results = reg.findall(text)
    numbers = [float(value) * mul for value in results]
    if len(numbers) == 1:
        numbers = numbers * 2
    elif len(numbers) == 0:
        numbers = [0, 0]
    return {
        'min_{}'.format(var): min(numbers),
        'max_{}'.format(var): max(numbers)
    }

In [5]:
# Define scraping functions for each attribute
def get_popularity(attribute):
    return attribute

def get_height(attribute):
    return attribute

def get_weight(attribute):
    return attribute

def get_expectancy(attribute):
    return attribute

def get_group(attribute):
    return attribute

# Create a dictionary mapping attribute names to their respective scraping functions
attr_function = {
    'akc_breed_popularity': get_popularity,
    'height': get_height,
    'weight': get_weight,
    'life_expectancy': get_expectancy,
    'group': get_group
}

# Function to extract attributes from a comment
def get_attributes_from_comment(comment):
    attributes_data = {}
    
    comment_content = BeautifulSoup(comment, 'html.parser')
    
    data_objects = comment_content.find_all('dataobject', type='document')
    
    for data_object in data_objects:
        attributes = data_object.find_all('attribute')
        
        for attribute in attributes:
            name = attribute.get('name')
            value = attribute.get_text(strip=True)
            
            # Check if the attribute name is in the attr_function dictionary
            if name in attr_function:
                scraping_function = attr_function[name]
                attributes_data[name] = scraping_function(value)
    
    return attributes_data

In [6]:
breed_attributes = []

def get_main_attributes(breed_soup):
    # Find all comments in the HTML
    comments = breed_soup.find_all(string=lambda text: isinstance(text, Comment))

    # Iterate through comments
    for comment in comments:
        comment_content = BeautifulSoup(comment, 'html.parser')

        if comment_content.find('pagemap'):
            breed_attributes = get_attributes_from_comment(comment)
    
    return breed_attributes

In [7]:
def get_care_info(breed_soup):
    titles = breed_soup.find_all(
        'h4', class_='bar-graph__title'
    )
    
    values = breed_soup.find_all(
        'div', class_='bar-graph__section'
    )
    
    categories = breed_soup.find_all(
        'div', class_='bar-graph__text'
    )
    
    care_dict = {}
    
    for (title, value, category) in zip (titles, values, categories):
        t = title.get_text().lower().replace(' ', '_')
        t = t[t.find('/') + 1:]
        care_dict[t + '_value'] = float(
            value['style'].split()[1].split('%')[0]
        ) / 100
        care_dict[t + '_category'] = category.get_text()
    
    return care_dict

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

class Breed:
    def __init__(self, breed_url):
        self.url = breed_url
        
        # Set up a headless Chrome browser
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run in headless mode (no GUI)
        driver = webdriver.Chrome(options=options)

        # Load the web page
        driver.get(breed_url)

        # Get the page source after content has loaded
        page_source = driver.page_source

        # Create a BeautifulSoup object from the page source
        breed_soup = BeautifulSoup(page_source, 'html.parser')

        # Scrape content as needed
        self.breed_info = {}
        self.breed_info['Description'] = get_description(breed_soup)
        self.breed_info['temperament'] = get_temperament(breed_soup)
        self.breed_info.update(get_main_attributes(breed_soup))
        self.breed_info.update(get_care_info(breed_soup))

        # Close the browser
        driver.quit()

    def get_breed_info(self):
        return self.breed_info


In [9]:
def get_data():
    page = requests.get('https://www.akc.org/dog-breeds/')
    soup = BeautifulSoup(page.content, 'html.parser')
    
    breed_select = soup.find('select', id='breed-search')

    breeds = []
    
    for tag in breed_select.children:
        if isinstance(tag, Tag):
            if 'value' in tag.attrs and tag['value']:
                breeds.append(tag)
                
#     print(breeds[0:2])
    
    breed_dict = {}
    
    for breed in breeds[0:1]:
        breed_name = breed.get_text()
        breed_value = breed['value']
        breed_info = Breed(breed_value).get_breed_info()
        breed_dict[breed_name] = breed_info
    
    print('breed_dict', breed_dict)
    
#     return breed_dict

In [10]:
breed_dict = get_data()

breed_dict {'Affenpinscher': {'Description': "Loyal, curious, and famously amusing, this almost-human toy dog is fearless out of all proportion to his size. As with all great comedians, it's the Affenpinscher's apparent seriousness of purpose that makes his antics all the more amusing.The Affen's apish look has been described many ways. They've been called 'monkey dogs' and 'ape terriers.' The French say diablotin moustachu ('mustached little devil'), and Star Wars fans argue whether they look more like Wookies or Ewoks.     Standing less than a foot tall, these sturdy terrier-like dogs approach life with great confidence. 'This isn't a breed you train, 'a professional dog handler tells us, 'He's like a human. You befriend him.'\x9d The dense, harsh coat is described as 'neat but shaggy' and comes in several colors; the gait is light and confident. They can be willful and domineering, but mostly Affens are loyal, affectionate, and always entertaining. Affen people say they love being o